# Playing Stochastic Multi-Armed Bandit Games

Code inspired by the [Pyma bandit library](https://www.di.ens.fr/~cappe/Code/PymaBandits/) by O. Cappé and A. Garivier and the [SMPyBandit library](https://github.com/SMPyBandits/SMPyBandits) of Lilian Besson.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import time

# Bandit specific functions and classes (given) 
import Arms as arm
from StochasticBandit import *
from BanditTools import * 
import BanditBaselines as alg

# I) Getting started with simple strategies

## 1) Creating a bandit environment 

Different arm classes are defined in `Arms.py` (you are welcome to add more!). All arms can be samples (i.e. produce random realization from their underlying distribution) using the "sample" method. 


In [ ]:
arm1 = arm.Bernoulli(0.7)
print("The mean of arm 1 is",arm1.mean)
print("A sample from arm 1 is",arm1.sample(),"\n")


arm2 = arm.Gaussian(1,0.5)
print("The variance of arm 2 is",arm2.variance)
print("A sample from arm 2 is",arm2.sample())

To define a multi-armed bandit (a `MAB` object), you need to input a list of such arms. Some functions in `StochasticBandits.py` also define directly some particular MAB objects. MAB object have a method named generateReward that takes as an input and arm and generate a random reward from the underlying distribution. 

In [ ]:
bandit1 = MAB([arm.Bernoulli(0.2), arm.Gaussian(0.9, 1), arm.TruncatedExponential(2, 1)])
nbArms = bandit1.nbArms

bandit2 = BernoulliBandit([0.3, 0.4, 0.5]) # directly creates a Bernoulli bandit from a vector of means


print("The means of bandit instance 1 are", bandit1.means)
print("The means of bandit instance 2 are", bandit2.means)

print("Generated reward from arm 0 in bandit 2:", bandit2.generateReward(2))

## 2) Playing a bandit strategy 

In `BanditBaselines.py` you will find the two simplest examples of bandit strategy that we saw in class: the greedy strategy (Follow The Leader), which always selects the empirical best arm, and the Uniform strategy, which always selects an arm uniformly at random. A bandit strategy is a class that has two important methods: 
* self.chooseArmToPlay() : select the next arm to play 
* self.receiveReward(a,rew) : update the strategy when arm a was drawn and the reward rew was observed

I encourage you to keep the same structure for the smarter bandit strategies that you will implement in this notebook : specifying these two methods (and of course selecting the right attributes for the class). 

The following function can be used to run a given bandit strategy on some bandit instance. 

In [ ]:
def OneBanditOneLearnerOneRun(bandit, strategy, timeHorizon):
    """
    Run a bandit strategy (strategy) on a MAB instance (bandit) for (timeHorizon) time steps
    output : sequence of arms chosen, sequence of rewards obtained
    """
    selections = []
    rewards = []
    strategy.clear() # reset previous history
    for t in range(timeHorizon):
        # choose the next arm to play with the bandit algorithm
        arm = strategy.chooseArmToPlay()
        # get the reward of the chosen arm
        reward = bandit.generateReward(arm)
        # update the algorithm with the observed reward
        strategy.receiveReward(arm, reward)
        # store what happened
        selections.append(arm)
        rewards.append(reward)
    return selections, rewards
    

We can use to it visualize what happen during a run of a bandit strategy, first what arms get selected. Don't hesitate to re-evaluate this cell multiple times, which will perform different random runs of the algorithm. You can also try to see what happens with the uniform strategy. 

In [ ]:
def ViewSelections(selections):
    plt.clf()
    nbBins=max(selections) + 1
    plt.xlabel("Arms", fontsize=14)
    plt.ylabel("Number of arms selections", fontsize=14)
    plt.hist(selections, nbBins)
    plt.title("Number of selections of each arm", fontsize=14)
    axes=plt.gca()
    axes.xaxis.set_ticks(range(nbBins))

In [ ]:
timeHorizon = 200
bandit = bandit2
nbArms = bandit.nbArms

strategy = alg.FTL(nbArms)
#strategy = alg.UniformExploration(nbArms)

# generating a random trajectory of the bandit algorithm
selections, rewards = OneBanditOneLearnerOneRun(bandit, strategy, timeHorizon)

# compute average reward
print("the means of the bandit are",bandit.means)
print("the average reward collected is",np.sum(rewards)/timeHorizon)

# Histogram of the number of selections
ViewSelections(selections)

The (expected) cumulative regret of a strategy $\mathcal{A}=(A_t)_{t\in \mathbb{N}^*}$ is defined as $$\mathcal{R}_T(\mathcal{A}) = \mathbb{E}\left[\sum_{t=1}^{T} (\mu^\star - \mu_{A_t})\right].$$
The random quantity inside the expectation is the cumulative regret on one simulation (sometimes called pseudo-regret), ${R}_T(\mathcal{A}) = \sum_{t=1}^{T} (\mu^\star - \mu_{A_t})$. 

**Write a function that returns a vector $({R}_t(\mathcal{A}))_{t \in [1,T]}$, which gives the cumulative (pseudo)-regret in each time step.** 

In [ ]:
def CumulativeRegret(bandit,selections):
    """Compute the pseudo-regret associated to a sequence of arm selections"""
    T = len(selections)
    meansB = np.array(bandit.means)
    mustar = max(meansB)
    return np.cumsum(mustar*np.ones(T)-meansB[selections])

You can now visualize the regret as a function of time. Trying on different runs, you will see that the regret of FTL is either constant, or linear in $t$. Its expected regret will thus be linear, as we shall see.

In [ ]:
bandit = bandit2
timeHorizon = 1000
strategy = alg.FTL(nbArms)
selec, rewards = OneBanditOneLearnerOneRun(bandit, strategy, timeHorizon)
regret = CumulativeRegret(bandit, selec)

plt.figure(2)
plt.clf()
plt.xlabel("Time steps", fontsize=14)
plt.ylabel("Cumulative regret", fontsize=14)
plt.title("Regret as a function of time")
plt.plot(range(timeHorizon), regret, 'black', linewidth=1)
 

## 3) Comparison of two (bad) algorithm on multiple runs

The regret is defined as an **expectation**, so we need **several runs** to estimate its value. We can also take a look at empirical distribution of the pseudo-regret. The function below gathers results accross multiple runs.

In [ ]:
def OneBanditOneLearnerMultipleRuns(bandit, strategy, timeHorizon, N_exp, tsave=[]):
    """
    Perform N_exp runs of a bandit strategy (strategy) on a MAB instance (bandit) for (timeHorizon) time steps 
    and compute the pseudo-regret of each run 
    optional : tsave is a vector of time steps in which the results will be stored (set to 1:timeHorizon by default)
    output : a table of size N_exp x |tsave| in which each row is the pseudo-regret at the sub-sampled times 
    """
    if (len(tsave) == 0):
        tsave = np.arange(timeHorizon)
    savedTimes = len(tsave)
    Regret = np.zeros((N_exp, savedTimes)) # Store the regret values on different runs
    for n in range(N_exp):
        np.random.seed()
        # run the bandit strategy
        selections, rewards = OneBanditOneLearnerOneRun(bandit, strategy, timeHorizon)
        # compute its pseudo-regret
        regret_one_run = CumulativeRegret(bandit, selections)
        # store (a sub-sampling of) the cumulative regret
        Regret[n, :] = np.array(regret_one_run)[tsave] 
    return Regret

In [ ]:
bandit = MAB([arm.Bernoulli(0.2), arm.Bernoulli(0.5), arm.Gaussian(0.6, 0.25), arm.TruncatedExponential(1, 1)])
bandit.means
print(bandit.means)

algo1 = alg.FTL(bandit.nbArms)
algo2 = alg.UniformExploration(bandit.nbArms)


### a) For one algorithm (FTL)

In [ ]:
N_exp = 200
timeHorizon = 500
tsave = np.arange(1, timeHorizon, 10) 
#print(tsave)
Regret = OneBanditOneLearnerMultipleRuns(bandit, algo1, timeHorizon, N_exp, tsave)

One can start by displaying the mean regret and some quantiles. 

In [ ]:
meanRegret = np.mean(Regret, 0)
upperQuantile = np.quantile(Regret, 0.95, 0) 
lowerQuantile = np.quantile(Regret, 0.05, 0)

plt.clf()
plt.plot(tsave, meanRegret, linewidth=3.0, color="b", label="mean regret for "+algo1.name())
plt.plot(tsave, upperQuantile, linestyle="dashed", color="b")
plt.plot(tsave, lowerQuantile, linestyle="dashed", color="b")
plt.legend()


#### b) FTL versus Uniform

In [ ]:
def RunExpes(algorithms,bandit,N_exp,timeHorizon,step=10,quantiles = "on"):
    '''run experiments for multiple algorithms and display their regret on the same plot'''
    plt.clf()
    tsave = np.arange(1,timeHorizon,step)
    colors = ["b","r","g","k","m"]
    for i in range(len(algorithms)):
        algo=algorithms[i]
        Regret = OneBanditOneLearnerMultipleRuns(bandit, algo, timeHorizon, N_exp, tsave)
        plt.plot(tsave, np.mean(Regret, 0), linewidth=2.0, color=colors[i], label="mean regret of "+ algo.name())
        if (quantiles == "on"):
            plt.plot(tsave, np.quantile(Regret, 0.95, 0), tsave, np.quantile(Regret,0.05,0), linestyle="dashed", color=colors[i])
    plt.legend()
    plt.show()
        

In [ ]:
N_exp = 200
timeHorizon = 500
tsave = np.arange(1, timeHorizon, 10)

RunExpes([algo1,algo2],bandit,N_exp,timeHorizon,10,"on")

As you can see, both algorithms have **linear regret**. The regret of FTL is smaller than that of the Uniform strategy, however its pseudo-regret has a much higher variance. 

# II) Implementing UCB


The UCB($\alpha$) algorithm that we saw in class selects in round $t$ the arm 
$$A_{t} = \underset{a}{\text{argmax}} \left[\hat{\mu}_a(t-1) + \sqrt{\frac{\alpha \log(t)}{N_a(t-1)}}\right],$$
where $\hat{\mu}_a(t)$ is the empirical mean of arm $a$ after $t$ rounds and $N_a(t)$ is the number of selections of arm $a$ till that time. 

**Implement UCB($\alpha$)**

In [ ]:
class UCB:
    """UCB1 with parameter alpha"""
    def __init__(self, nbArms, alpha=1/2):
        self.nbArms = nbArms
        self.alpha = alpha
        self.clear()

    def clear(self):
        self.nbDraws = np.zeros(self.nbArms)
        self.cumRewards = np.zeros(self.nbArms)
        self.t = 0
    
    def chooseArmToPlay(self):
        # TO BE COMPLETED
        pass

    def receiveReward(self, arm, reward):
        self.t = self.t + 1
        self.nbDraws[arm] = self.nbDraws[arm] + 1
        self.cumRewards[arm] = self.cumRewards[arm] + reward

    def name(self):
        return "UCB"

Observe that UCB with $\alpha = 0$ coincides with Follow the Leader and has therefore linear regret, whereas $\alpha = 1/2$ always guarantees logarithmic regret on every bandit instance with bounded rewards. Still, on a particular instance it is possible to find a value of $\alpha$ that leads to better performance compared to $\alpha = 1/2$ (but it will not work on *any* problem instance). 

**On a bandit problem of your choice run UCB with different values of $\alpha$ and find a good value for this instance**

# III) Thompson Sampling for Bernoulli bandits 

    For Bernoulli bandit, we saw in class that UCB is not the best possible algorithm, and the goal of this part is to implement a better algorithm for Bernoulli rewards: Thompson Sampling, which is a Bayesian algorithm. 

Letting $\pi_a(t)$ denote the posterior distribution on arm $a$ after $t$ rounds, Thompson Sampling can be implemented as follows: 

$$\forall a, \theta_a(t) \sim \pi_a(t-1), \ \ \ A_{t} = \underset{a}{\text{argmax }} \theta_a(t)$$

In the Bernoulli case, with a uniform (or more generally a Beta$(\alpha,\beta)$) prior distribution on each mean, the posterior distribution remains a Beta distribution. More precisely, one has

$$\pi_a(t) = \mathrm{Beta}\left(\alpha + S_a(t), \beta + N_a(t) - S_a(t)\right),$$

where $S_a(t)$ is the sum of rewards received from arm $a$ after $t$ rounds, and $N_a(t)$ is the number of selections of that arm, as before.

**Implement Thompson Sampling for Bernoulli bandits with a Beta prior.** 


In [ ]:
class ThompsonSampling:
    """Thompson Sampling with Beta(a,b) prior and Bernoulli rewards"""
    pass

**Compare the regret of UCB and Thompson Sampling on the following two bandit instances**

In the first bandit, the means are all close to $1/2$, whereas in the second the means are all very low, which corresponds to what typically happens in applications to online advertisement. Comment on the results. 

In [ ]:
banditA = BernoulliBandit([0.45, 0.5, 0.6])  
banditB = BernoulliBandit([0.1, 0.05, 0.02, 0.01])

* Comparison on bandit A

* Comparison on bandit B

Thompson Sampling always outperforms UCB, but the difference is much more striking when the means are low.

**In a bandit instance for which the rewards are now arbitrary bounded distributions supported in $[0,1]$ (e.g. truncated exponentials), can you think of an adaptation of the (binary) Thompson Sampling with the same regret guarantees?** 

# IV) Stochastic versus adversarial algorithms 


The EXP3 algorithm that we saw in class can be used to solve a stochastic bandit problem with rewards in $[0,1]$, by defining an appropriate loss function. 

**How do you define the loss function, and what upper bound do we have on the expected regret $R_T(\text{EXP3})$?**


**Implement EXP3 for stochastic bandits, and compare it to Thompson Sampling. For what kind of instances could this algorithm perform best?**